## Mode of Simulation 

Three type of simulation 

<strong>1. Profit maximization</strong> 

<strong>2. Solo Rides</strong>

<strong>3. Pooled Rides</strong>

## Load ExMAS and MaaSSim


In [24]:
%load_ext autoreload
%autoreload 2
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from MaaSSim.utils import get_config, load_G, prep_supply_and_demand, generate_demand, generate_vehicles, initialize_df  # simulator
from MaaSSim.data_structures import structures as inData
from MaaSSim.simulators import simulate
from MaaSSim.visualizations import plot_veh
from MaaSSim.shared import prep_shared_rides
import logging
import matplotlib.pyplot as plt

import pandas as pd
import ExMAS

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Delft, Netherlands

In [25]:
params = get_config('../../data/config/delft.json')  # load configuration

params.times.pickup_patience = 3600 # 1 hour of simulation
params.simTime = 1 # 6 minutes hour of simulation
params.nP = 50 # reuqests (and passengers)
params.nV = 20 # vehicles


## Parameters for ExMAS

In [26]:
params.t0 = pd.Timestamp.now()
params.shareability.avg_speed = params.speeds.ride
params.shareability.shared_discount = 0.25
params.shareability.delay_value = 1
params.shareability.WtS = 1.3
params.shareability.price = 1.5 #eur/km
params.shareability.VoT = 0.0035 #eur/s
params.shareability.matching_obj = 'u_veh' #minimize VHT for vehicles
params.shareability.pax_delay = 0
params.shareability.horizon = 600
params.shareability.max_degree = 4
params.shareability.nP = params.nP
params.shareability.share = 1
params.shareability.without_matching = True
params.shareability.operating_cost = 0.0005
params.shareability.comm_rate = 0.2

inData = load_G(inData, params)  # load network graph 

inData = generate_demand(inData, params, avg_speed = False)
inData.vehicles = generate_vehicles(inData,params.nV)
inData.vehicles.platform = inData.vehicles.apply(lambda x: 0, axis = 1)
inData.passengers.platforms = inData.passengers.apply(lambda x: [0], axis = 1)
inData.requests['platform'] = inData.requests.apply(lambda row: inData.passengers.loc[row.name].platforms[0], axis = 1) 
inData.platforms = initialize_df(inData.platforms)
inData.platforms.loc[0]=[1,'Uber',30]
params.shareability.share = 1
params.shareability.without_matching = True



# params.kpi = 1 (Profit Maximazation)
# params.kpi = 2 (Solo Rides)
# params.kpi = 3 (Pooled Rides)


In [27]:
params.kpi = 1


### Profit Mazimization - Begin 

In [28]:
#inData = ExMAS.main(inData, params.shareability) # create shareability graph (ExMAS) 
inData = ExMAS.main(inData, params.shareability, plot = False)

20-07-23 12:24:08-INFO-Initializing pairwise trip shareability between 50 and 50 trips.
20-07-23 12:24:08-INFO-creating combinations
20-07-23 12:24:09-INFO-2450	 nR*(nR-1)
20-07-23 12:24:09-INFO-Reduction of feasible pairs by 98.78%
20-07-23 12:24:09-INFO-Degree 2 	Completed
20-07-23 12:24:09-INFO-trips to extend at degree 2 : 76
20-07-23 12:24:09-INFO-At degree 2 feasible extensions found out of 5 searched
20-07-23 12:24:09-INFO-Degree 3 	Completed
20-07-23 12:24:09-INFO-trips to extend at degree 3 : 5
20-07-23 12:24:09-INFO-At degree 3 feasible extensions found out of 0 searched
20-07-23 12:24:09-INFO-Degree 4 	Completed
20-07-23 12:24:09-INFO-Max degree reached 4
20-07-23 12:24:09-INFO-Trips still possible to extend at degree 4 : 0


In [29]:
inData = prep_shared_rides(inData, params.shareability) # prepare schedules

20-07-23 12:24:10-INFO-Matching 50 trips to 131 rides in order to min u_veh
20-07-23 12:24:10-INFO-Problem solution: Optimal. 
Total costs for single trips:         14,045 
reduced by matching to:               11,953


In [30]:
inData.sblts.rides

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,row,selected,nodes,req_id,sim_schedule,ttrav,dist,fare,commission,driver_revenue
0,[0],3.018500,163,1,[3.0185],"[0, 163]",[0],[0],1,0,...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1620149364, 4217813217]","[None, 0, 0]",node time req_id od 0 ...,163,1.63,2.448000,0.489600,1.9584
1,[1],6.128000,331,1,[6.128],"[69, 331]",[1],[1],1,1,...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 3684805925, 1569858134]","[None, 1, 1]",node time req_id od 0 ...,331,3.31,4.969500,0.993900,3.9756
2,[2],8.869000,479,1,[8.869],"[146, 479]",[2],[2],1,2,...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1410536895, 1391417164]","[None, 2, 2]",node time req_id od 0 ...,479,4.79,7.192500,1.438500,5.7540
3,[3],10.128500,547,1,[10.1285],"[155, 547]",[3],[3],1,3,...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44884118, 44731083]","[None, 3, 3]",node time req_id od 0 NaN...,547,5.47,8.214000,1.642800,6.5712
4,[4],2.904500,157,1,[2.9045],"[167, 157]",[4],[4],1,4,...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 565046998, 44847009]","[None, 4, 4]",node time req_id od 0 N...,157,1.57,2.355000,0.471000,1.8840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,"[2, 3, 8]",21.918550,964,32,"[7.978775000000001, 9.40465, 4.535125000000001]","[14.0, 273, 144, 138, 210, 199]","[2, 3, 8]","[2, 8, 3]",3,126,...,"[0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",1,"[None, 1410536895, 44884118, 3790294279, 13914...","[None, 2, 3, 8, 2, 8, 3]",node time req_id od 0 ...,964,9.64,14.429250,2.885850,11.5434
127,"[1, 3, 5]",19.341250,875,32,"[5.8565249999999995, 10.03255, 3.4521750000000...","[79.0, 66, 247, 111, 142, 309]","[1, 3, 5]","[1, 5, 3]",3,127,...,"[0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",0,"[None, 3684805925, 44884118, 44827614, 1569858...","[None, 1, 3, 5, 1, 5, 3]",node time req_id od 0 ...,875,8.75,12.152250,2.430450,9.7218
128,"[27, 29, 30]",14.181975,515,32,"[6.360200000000001, 4.491825, 3.32995]","[1885.0, 104, 77, 208, 43, 83]","[27, 29, 30]","[29, 30, 27]",3,128,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44878925, 44861853, 44875431, 44813589,...","[None, 27, 29, 30, 29, 30, 27]",node time req_id od 0 ...,515,5.15,9.145125,1.829025,7.3161
129,"[31, 29, 34]",16.639725,570,32,"[7.724774999999999, 4.614675, 4.300275]","[1915.0, 146, 80, 212, 53, 79]","[31, 29, 34]","[29, 34, 31]",3,129,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 2804814348, 44861853, 44875057, 4481358...","[None, 31, 29, 34, 29, 34, 31]",node time req_id od 0 ...,570,5.70,10.920375,2.184075,8.7363


In [31]:
sim = simulate(params = params, inData = inData, logger_level = logging.CRITICAL) # simulate

20-07-23 12:24:13-INFO-Matching 50 trips to 131 rides in order to min u_veh
20-07-23 12:24:13-INFO-Problem solution: Optimal. 
Total costs for single trips:         14,045 
reduced by matching to:               11,953
0.0005
20-07-23 12:24:16-CRITICAL-this is request 0 with [0, 55] available rides.
20-07-23 12:24:16-CRITICAL-ride 0 available [1]
20-07-23 12:24:16-CRITICAL-ride 55 available [1, 0]
20-07-23 12:24:16-CRITICAL-this is request 0 with [0, 55] still available rides.
Profit Maximization - Pick highest fare
20-07-23 12:24:16-CRITICAL-vehicle 1 has 2 choices
0.0005
20-07-23 12:24:16-CRITICAL-this is request 1 with [1, 56, 67, 102, 103, 120, 127] available rides.
20-07-23 12:24:16-CRITICAL-ride 1 available [1]
20-07-23 12:24:16-CRITICAL-ride 56 available [1, 0]
20-07-23 12:24:16-CRITICAL-ride 67 available [1, 0]
20-07-23 12:24:16-CRITICAL-ride 102 available [1, 0]
20-07-23 12:24:16-CRITICAL-ride 103 available [1, 0]
20-07-23 12:24:16-CRITICAL-ride 120 available [0, 1]
20-07-23 12

In [32]:
sim.res[0].veh_exp

,nRIDES,nREJECTED,OUT,STARTS_DAY,OPENS_APP,RECEIVES_REQUEST,ACCEPTS_REQUEST,REJECTS_REQUEST,IS_ACCEPTED_BY_TRAVELLER,IS_REJECTED_BY_TRAVELLER,...,STARTS_REPOSITIONING,REPOSITIONED,DECIDES_NOT_TO_DRIVE,ENDS_SHIFT,TRAVEL,WAIT,CRUISE,OPERATIONS,IDLE,PAX_KM
1,3.0,0.0,False,0,0,974.0,45.0,0,60.0,0,...,0,0,0,8273.0,767.0,0,590.0,195.0,6721.0,7.684
2,2.0,0.0,False,0,0,1890.0,30.0,0,40.0,0,...,0,0,0,7875.0,572.0,0,332.0,130.0,6841.0,5.733
3,4.0,0.0,False,0,0,1714.0,30.0,0,268.0,0,...,0,0,0,7807.0,521.0,0,339.0,418.0,6529.0,11.277
4,2.0,0.0,False,0,0,2362.0,30.0,0,40.0,0,...,0,0,0,7475.0,482.0,0,350.0,130.0,6513.0,4.835
5,4.0,0.0,False,0,0,621.0,21.0,0,76.0,0,...,0,0,0,8392.0,938.0,0,631.0,217.0,6606.0,16.541
6,4.0,0.0,False,0,0,1977.0,60.0,0,80.0,0,...,0,0,0,7328.0,969.0,0,265.0,260.0,5834.0,9.705
7,2.0,0.0,False,0,0,1458.0,15.0,0,241.0,0,...,0,0,0,7972.0,606.0,0,447.0,316.0,6603.0,9.976
8,4.0,0.0,False,0,0,1507.0,45.0,0,80.0,0,...,0,0,0,8172.0,442.0,0,433.0,245.0,7052.0,6.615
9,1.0,0.0,False,0,0,956.0,15.0,0,20.0,0,...,0,0,0,9063.0,441.0,0,274.0,65.0,8283.0,4.419
10,2.0,0.0,False,0,0,2392.0,15.0,0,28.0,0,...,0,0,0,7371.0,500.0,0,441.0,95.0,6335.0,8.946


In [33]:
sim.vehs[1].rdf

""


In [34]:
pd.DataFrame(sim.vehs[1].myrides)

,veh,pos,t,event,paxes
0,1,4.471882e+07,0,STARTS_DAY,[]
1,1,4.471882e+07,0,OPENS_APP,[]
2,1,4.471882e+07,0,RECEIVES_REQUEST,[]
3,1,4.471882e+07,15,ACCEPTS_REQUEST,[]
4,1,4.471882e+07,35,IS_ACCEPTED_BY_TRAVELLER,[None]
5,1,1.620149e+09,491,ARRIVES_AT_PICKUP,[]
6,1,1.620149e+09,491,MEETS_TRAVELLER_AT_PICKUP,[]
7,1,1.620149e+09,521,DEPARTS_FROM_PICKUP,[0]
8,1,4.217813e+09,684,ARRIVES_AT_DROPOFF,[0]
9,1,4.217813e+09,1627,RECEIVES_REQUEST,[]


In [26]:
responses = []
avg_kpi = []
idle_time = []

for i in range(2, 3):
    params.kpi = i
    sim = simulate(params = params, inData = inData, logger_level = logging.CRITICAL) # simulate
    sim.res[0].veh_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/veh{}.csv'.format(i))
    sim.res[0].pax_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/pax{}.csv'.format(i))
    #driver_data.loc['Cost'] = driver_data.loc['Revenue'].apply(lambda x: x*params.shareability.operating_cost)['Vehicles'] = sim.res[0].veh_exp.index
    #sim.res[0].veh_exp['ds'] = f"{i}"
    
    #responses.append(sim.res[0].veh_exp)
     
   # vehicles = sim.res[0].veh_exp.loc[sim.res[0].veh_exp["nRIDES"] > 0]
    #no_of_veh = len(vehicles)
    
    #avg_kpi.append(sim.res[0].all_kpi/no_of_veh)
    #idle_time.append(vehicles['IDLE'].sum()/no_of_veh)
    
    #_inData.sblts.rides['commission'] = _inData.sblts.rides.apply(lambda row: row.fare*sp.comm_rate, axis=1)
    

0.0005
20-07-23 11:49:01-CRITICAL-this is request 0 with [0] available rides.
20-07-23 11:49:01-CRITICAL-ride 0 available [1]
20-07-23 11:49:01-CRITICAL-this is request 0 with [0] still available rides.
0.0005
20-07-23 11:49:01-CRITICAL-this is request 1 with [1] available rides.
20-07-23 11:49:01-CRITICAL-ride 1 available [1]
20-07-23 11:49:01-CRITICAL-this is request 1 with [1] still available rides.
0.0005
20-07-23 11:49:02-CRITICAL-this is request 2 with [2, 76, 77, 78, 79, 80] available rides.
20-07-23 11:49:02-CRITICAL-ride 2 available [1]
20-07-23 11:49:02-CRITICAL-ride 76 available [1, 0]
20-07-23 11:49:02-CRITICAL-ride 77 available [1, 0]
20-07-23 11:49:02-CRITICAL-ride 78 available [1, 0]
20-07-23 11:49:02-CRITICAL-ride 79 available [1, 0]
20-07-23 11:49:02-CRITICAL-ride 80 available [1, 0]
20-07-23 11:49:02-CRITICAL-this is request 2 with [2, 76, 77, 78, 79, 80] still available rides.
Pooled rides
20-07-23 11:49:02-CRITICAL-vehicle 6 has 6 choices
0.0005
20-07-23 11:49:02-CR